In [1]:
!pip install opencv-python
!pip install matplotlib
!pip install numpy
!pip install glob

You are using pip version 9.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Could not find a version that satisfies the requirement glob (from versions: )
No matching distribution found for glob
You are using pip version 9.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob
import shutil

In [3]:
# Defining Constants

# Chroma Key Threshold for Video 1
lower_green = np.array([10, 110, 10])
upper_green = np.array([90, 190, 90])

# Chroma Key Threshold for Video 2
lower_white = np.array([210, 210, 210])
upper_white = np.array([255, 255, 255])

In [4]:
# Creating Output Directories
# if not os.path.exists('output'):
shutil.rmtree('./output') 
os.makedirs('output')
os.makedirs('output/task_1')
os.makedirs('output/task_1/a')
os.makedirs('output/task_1/b')
os.makedirs('output/task_2')
os.makedirs('output/task_3')
    

In [5]:
import re

def tryint(s):
    try:
        return int(s)
    except:
        return s

def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)
    return l

In [6]:
# Function to read image
def read_image(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [7]:
# Function to extract out mask for Chroma Key
def extract_mask(img, low_thresh, high_thresh):
    mask = cv2.inRange(img, low_thresh, high_thresh)
    masked_img = np.copy(img)
    masked_img[mask != 0] = [0, 0, 0]
    return masked_img, mask

In [8]:
# Function to cut foreground mask shape in the background video
def mask_background(bg_img, mask):
    bg_img[mask == 0] = [0, 0, 0]
    return bg_img

In [9]:
# Function to merge foreground and background after Chroma Keying
def merge_bg_fg(fg, bg):
    final_img = fg + bg
    return final_img

In [28]:
# Function to process a video and apply Chroma Keying
def apply_chroma_key(fg_path, bg_path, lower_thresh, upper_thresh):
    cap_fg = cv2.VideoCapture(fg_path)
    cap_bg = cv2.VideoCapture(bg_path)

    if cap_fg.isOpened() == False:
        print("Error opening video file")

    if cap_bg.isOpened() == False:
        print("Error opening video file")

    frames = []
    masked_imgs = []
    masked_bgs = []
    masks = []

    while cap_fg.isOpened() and cap_bg.isOpened():
        ret_fg, fg = cap_fg.read()
        ret_bg, bg = cap_bg.read()

        if ret_fg == True and ret_bg == True:

            masked_img, mask = extract_mask(fg, lower_thresh, upper_thresh)
            masked_bg = mask_background(bg, mask)
            final_img = merge_bg_fg(masked_img, masked_bg)
            frames.append(final_img)
            masked_imgs.append(masked_img)
            masked_bgs.append(masked_bg)
            masks.append(mask)

            if cv2.waitKey(25) & 0xFF == ord('q'):
                break

        else:
            break
    
    cap_fg.release()
    cap_bg.release()

    return frames #, masked_imgs, masked_bgs, masks

In [11]:
# Function to combine frames into video
def convert_frames_to_video(frames, output_file, fps): 
    size =  frames[0].shape
    size = (size[1], size[0])
    out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frames)):
        out.write(frames[i])
    out.release()


In [12]:
# Function to convert a video into images
def convert_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    
    if cap.isOpened() == False:
        print("Error opening video file")

    frames = []

    while cap.isOpened():
        ret, frame = cap.read()

        if ret == True:
            frames.append(frame)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break

        else:
            break
    
    cap.release()
    return frames    

In [13]:
def web_cam_feed(path_to_save):
    cap = cv2.VideoCapture(0)

    i=0
    while(True):
        ret, frame = cap.read()

        if ret == True:
            cv2.imshow('Video Camera Feed'.format(i), frame)
            cv2.imwrite(filename=path_to_save+'{}.jpg'.format(i), img=frame)
            i+=1
        else:
            break

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break 

    cap.release()
    cv2.destroyAllWindows()

In [22]:
def display_video(video_path):
    cap = cv2.VideoCapture(video_path)

    i=0
    while(True):
        ret, frame = cap.read()

        if ret == True:
            cv2.imshow(video_path, frame)
        else:
            break

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break 

    cap.release()
    cv2.destroyAllWindows()

### Task 1 - Video <-> Image

#### Task 1.a - Convert Video to Constituent Frames

In [14]:
def task_1_a():
    frames = convert_video_frames('./resources/sample_video.mp4')
    
    i=1
    for frame in frames:
        cv2.imwrite("./output/task_1/a/{}.png".format(i), frame) 
        i+=1
        
    print("Output saved in ./output/task_1/a")
    
task_1_a()

Output saved in ./output/task_1/a


#### Task 1.b - Convert Frames to Video

In [15]:
def task_1_b():
    files = sort_nicely(glob.glob( "./output/task_1/a/*.png"))
    frames = []
    for file in files:
        frames.append(cv2.imread(file))
    convert_frames_to_video(frames, "./output/task_1/b/combined.mp4", 24)
    
    print("Output saved in ./output/task_1/b/combined.mp4")
    
task_1_b()

Output saved in ./output/task_1/b/combined.mp4


# Task 2 - Take input from camera and save the frames

In [16]:
print("Press Q to quit the camera feed")

def task_2():
    web_cam_feed('./output/task_2/')
    print("Output saved in ./output/task_2/")

task_2()

Press Q to quit the camera feed
Output saved in ./output/task_2/


### Task 3 - Apply Chroma Key to a sample video and a custom video

#### Task 3.a - Sample Video

In [29]:
def task_3_a():
    display_video('./resources/sample_fg.mp4')
    display_video('./resources/sample_bg.mp4')
    frames = apply_chroma_key('./resources/sample_fg.mp4', './resources/sample_bg.mp4', lower_green, upper_green)
    convert_frames_to_video(frames, './output/task_3/sample_final.mp4', 24) 
#     convert_frames_to_video(a, './output/task_3/a.mp4', 24) 
#     convert_frames_to_video(b, './output/task_3/b.mp4', 24) 
#     convert_frames_to_video(c, './output/task_3/c.mp4', 24) 
    display_video('./output/task_3/sample_final.mp4')

task_3_a()

#### Task 3.b - Custom Video

In [30]:
def task_3_b():
    display_video('./resources/custom_fg.mp4')
    display_video('./resources/custom_bg.mp4')
    frames = apply_chroma_key('./resources/custom_fg.mp4', './resources/custom_bg.mp4', lower_white, upper_white)
    convert_frames_to_video(frames, './output/task_3/custom_final.mp4', 24) 
    display_video('./output/task_3/custom_final.mp4')
    
task_3_b()